# Labeling Wine by Clustering

Today people can be inundated with data. This can be a gigantic problem for firms; by overwhelming customers, companies can lose sales. Add in the fact that the current generation is known for having a short attention span and you have a recipe for losing customers if your marketing strategy is not concise and catchy. 

Now this notebook looks at a particular product: wine. If you are a wine connoisseur, I recommend you stop reading now; this notebook may rub you the wrong way. For everyone else, wine can be overly complicated. Thousands of diffrent attributes can be used to describe wine and some, honestly, just sound like fluff. This notebook will look at a particular marketing problem when it comes to describing many different wines to customers.

Basically, how to catergize wine into dinsticnt bins based on all their different attributes. By doing this effectively, a wine retailer can reduce the cost associated with all the data around different wines, helping the customer buy faster and enjoy sooner. Also, if they do it really well, customers may come to rely on these bins and use them consistly into their future (making these labels mainstream and adding other avenues to capitalize on).

## The Technical Aspects

Because the problem is to label/catergize wines, and yet the catergories don't yet exist; this problem falls into the unsupervized learning realm. Becasue of this, deciding on whether the clusters created from the data makes sense becomes very, very, very subjective. I won't comment on whether the methods used throughout this notebook are good or not and will leave that up to reader. 

Anyway, the plan is to try several different techniques to catergorize: <b> K-means, Density-Based Spatial Clustering of Applications with Noise (DBSCAN), and Topic Models</b>. The notebook will also use some supervised learning (mainly regression) to aid in feature enginerring. Let's start with the data.

In [146]:
#import packages
import graphlab as gl
import re
import math

In [147]:
#load data into Sframe
wine=gl.SFrame('./wine.gl/')

In [148]:
#load head of wine table
wine.head()

id,name,price,sales,img,reviews
0,Jamieson Ranch VineyardsDouble Lariat Cabernet ...,60.0,34.99,http://cache.wine.com/labels/147264l.jpg ...,"{'anonymous1914596960':('', '4')} ..."
1,Rombauer Chardonnay 2014,36.0,34.99,no pic,{'hgoodstein': ('Lovethis chardonnay! Best ...
2,Tenuta San GuidoSassicaia 2012 ...,235.0,189.0,no pic,None
3,Clarendon Hills SandownCabernet Sauvignon 2010 ...,45.0,29.99,no pic,"{'anonymous1431286300':('', '2'), ..."
4,Silver Oak AlexanderValley Cabernet Sauvi ...,45.0,None,no pic,"{'RedPill': ('', '1'),'sherin_sruthi': ('', ..."
5,Chateau Lafite-Rothschild2000 ...,None,None,http://cache.wine.com/labels/42783l.jpg ...,"{'DrDon': ('', '5'),'jeevasurya1': ('', ' ..."
6,Clos du Val CabernetSauvignon 2013 ...,38.0,29.99,no pic,"{'anonymous346739964':('', '5'), ..."
7,Valdicava Brunello diMontalcino 2010 ...,75.0,None,no pic,None
8,Duckhorn Napa ValleyCabernet Sauvignon 2012 ...,72.0,69.99,no pic,None
9,Stag's Leap Wine CellarsCask 23 Cabernet ...,165.0,None,no pic,"{'anonymous519829852':('', '1')} ..."


## Cleaning the Data

The wine data for this notebook has been collected by scraping a large online wine seller. The information collected is the wine name, price, picture, reviews of wine, wine maker's notes on wine, summary about winery, location of winery, year, past awards, current awards, similar wines, style, and customer reviews (see frame above for reference). As with any data science project the information is messy and needs to be cleaned (needs to convert things into a usable form; this could mean dropping, appending, combining, etc.) Anyway, we will not show the 'Fun and Exciting' cleaning procedure here and instead will show cleaned frames for the specified technique to be used.

## DBSCAN 

DSBCAN, short for Density-Based Spatial Clustering of Applications with Noise, is the most popular density-based clustering method. Density-based clustering algorithms attempt to capture our intuition that a cluster — a difficult term to define precisely — is a region of the data space where there are lots of points, surrounded by a region where there are few points. 

<b>DBSCAN</b> does this by partitioning the input data points into three types:
<ul><li>Core points have a large number of other points within a given neighborhood</li>
<li>Boundary points are within distance radius of a core point, but don't have sufficient neighbors of their own to be considered core.</li>
<li>Noise points comprise the remainder of the data. These points have too few neighbors to be considered core points, and are further than distance radius from all core points.</li>
</ul>

Clusters are formed by connecting core points that are neighbors of each other, then assigning boundary points to their nearest core neighbor's cluster. Noise points are left unassigned. DBSCAN has the following advantages over K-means: clusters do not need to be know a priori, can find more flexible shapes comapred to the sphereical shapes in k-means, and finds and labels outliers.

### First attempt with DBSCAN on data

In [149]:
#load cleaned dataframe for dbscan
sf=gl.SFrame('./fordbscanbasic.gl/')

#drop words wine & fruit
sf['tfidf']=sf['tfidf'].dict_trim_by_keys(['fruit','wine', 'winemaker','notes'])

In [150]:
sf.head()

year,lat,long,ln_price,wmr,tfidf
2013,38.220417,-122.228952,4.09434456222,winemaker s notesenticing aromas jump out ...,"{'tannins':2.992188494032592, ..."
2014,38.546127,-122.49276,3.58351893846,winemaker s notes juicypeaches pineapple and ...,"{'alluring':8.453879052391954, ..."
2012,43.225883,10.563555,5.45958551414,winemaker s notes intenseconcentrated and deep ...,"{'breadth':5.3929930956596195, ..."
2010,-34.919913,138.654506,3.80666248977,winemaker s notes theclarendon hills sandown ...,"{'tannins':1.9947923293550611, ..."
2011,38.741804,-122.955029,3.80666248977,winemaker s notes thealexander valley cabe ...,"{'currants':6.68012535905623, ..."
2000,45.224217,-0.772444,3.96289038246,winemaker s notes arelatively early growing ...,"{'precise':3.845925513951977, ..."
2013,38.387141,-122.313005,3.63758615973,winemaker s notes thiswine delivers everything ...,"{'tannins':2.992188494032592, ..."
2010,43.077656,11.491754,4.31748811354,winemaker s notesvaldicava philosophy is ...,"{'traditions':7.362433742125127, ..."
2012,38.529416,-122.490612,4.27666611902,winemaker s notes bothgenerous and graceful ...,"{'tannins':1.9947923293550611, ..."
2012,38.398864,-122.325167,5.1059454739,winemaker s notes thecask combines the ...,"{'years':3.642340393933401, ..."


The data we are going to use is location, log of price (to deal with the skewness), and tf-idf of the combined wine maker's notes, winery info, and reviews. (Note: the tf-idf column has been trimmed to drop the common words like: the, a, at, in, etc.)

In [151]:
# perform dbscan uisng graphlab create
# set dist
comp_dis=[[['year', 'lat', 'long'], 'euclidean', 1],[['ln_price'], 'euclidean', .75],[['tfidf'], 'cosine', 2]]

# run dbscan
dbscan_model = gl.dbscan.create(sf,distance=comp_dis, radius=5)
dbscan_model.summary() 

Starting brute force nearest neighbors model training.

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

...

| 72525                       |

| 0                           |

+-----------------------------+

[INFO] graphlab.toolkits.clustering.dbscan: Processing boundary points.


Class                                   : DBSCANModel

Schema
------
Number of examples                      : 9454
Number of feature columns               : 5
Max distance to a neighbor (radius)     : 5
Min number of neighbors for core points : 10
Number of distance components           : 3

Training summary
----------------
Total training time (seconds)           : 32.2594
Number of clusters                      : 14

Accessible fields
-----------------
cluster_id                              : Cluster label for each row in the input dataset.


## Results of first DBSCAN

DBSCAN resulted 14 clusters for a radius of 5 and notice that we weighed the features used differently, giving tf-idf the most wait. Now, let's see how it looks. To do that, the notebook will use a python package called wordclouds and create a table with particualr information to look at.

In [152]:
#to match id with wine
cur=gl.SFrame('./formodel.gl/')

names=cur[['id', 'name']]
names=names.add_row_number('row_id')
sf=sf.add_row_number('row_id')
sf=sf.join(names, on='row_id', how='left')
sf=sf.join(dbscan_model['cluster_id'], on='row_id', how='left')

sf[['id','name','cluster_id', 'type']].head()

id,name,cluster_id,type
0,Jamieson Ranch VineyardsDouble Lariat Cabernet ...,0,core
1,Rombauer Chardonnay 2014,0,core
2,Tenuta San GuidoSassicaia 2012 ...,2,core
3,Clarendon Hills SandownCabernet Sauvignon 2010 ...,4,core
4,Silver Oak AlexanderValley Cabernet Sauvi ...,0,core
5,Chateau Lafite-Rothschild2000 ...,2,core
6,Clos du Val CabernetSauvignon 2013 ...,0,core
7,Valdicava Brunello diMontalcino 2010 ...,2,core
8,Duckhorn Napa ValleyCabernet Sauvignon 2012 ...,0,core
9,Stag's Leap Wine CellarsCask 23 Cabernet ...,0,core


In [153]:
import pylab

#word cloud function
def plot_wordcount(cluster_id, model):
    from wordcloud import WordCloud
    cluster_data = sf.filter_by(model['cluster_id']['row_id'][model['cluster_id']['cluster_id']==cluster_id],'row_id')
    #print cluster_data['tfidf'].sketch_summary()
    #print cluster_data['name']
    data = list(cluster_data['tfidf'])
    data = [' '.join(c) for c in data]
    data = ' '.join(data)
    wc = WordCloud().generate(data) 
    figure(figsize=(20,10))
    return imshow(wc)

#to save multiple
i=0
while i<14:
    plot_wordcount(i,dbscan_model)
    pylab.savefig('cluster'+str(i)+'png')
    i+=1

## Looking at the results

We took out some words that were common to all the tf-idf columns, but even with that there still exists alot of similarity between the clusters. Btw, the wordclouds are showing you the most common words in the clusters, where size indicates the word is more prevalent. 
<center><table>
<tr><td><img src="cluster0png.png"></td>
<td><img src="cluster1png.png"></td>
<td><img src="cluster2png.png"></td></tr><br>
<tr><td><img src="cluster3png.png"></td>
<td><img src="cluster4png.png"></td>
<td><img src="cluster5png.png"></td></tr><br>
<tr><td><img src="cluster6png.png"></td>
<td><img src="cluster7png.png"></td>
<td><img src="cluster8png.png"></td></tr><br>
<tr><td><img src="cluster9png.png"></td>
<td><img src="cluster10png.png"></td>
<td><img src="cluster11png.png"></td></tr><br>
<tr><td><img src="cluster12png.png"></td>
<td><img src="cluster13png.png"></td>
</tr><br>
</table></center>

Now let's get some basic facts on clusters

In [154]:
#create table for info on dbscan
import csv
filenamed= 'tabledbscan'
titles=['HOW MANY WINES IN THE CLUSTER','mean price', 'min price', 'max price','upleft: lat long:', 'upright: lat long: ','downleft: lat long: ', 'downright: lat long: ']

writer = csv.writer(open(filenamed, 'a'))
writer.writerow(titles)   
 
#to get min, max, mean price
def getstats(cluster_id,model):
     cd = sf.filter_by(model['cluster_id']['row_id'][model['cluster_id']['cluster_id']==cluster_id],'row_id')
     holder=[]
     holder.append(str(len(cd['lat'])))
     holder.append(str(math.exp(cd['ln_price'].mean())))
     holder.append(str(math.exp(cd['ln_price'].min())))
     holder.append(str(math.exp(cd['ln_price'].max())))
     holder.append(str(cd['lat'].max()) +' '+str(cd['long'].min()))
     holder.append(str(cd['lat'].max()) +' '+str(cd['long'].max()))
     holder.append(str(cd['lat'].min()) +' '+str(cd['long'].min()))
     holder.append(str(cd['lat'].min()) +' '+str(cd['long'].max()))
     writer = csv.writer(open(filenamed, 'a'))
     writer.writerow(holder)   
        
#to loop through clusters
i=0
while i<14:
    getstats(i,dbscan_model)
    i+=1

In [161]:
#load table results
res=gl.SFrame.read_csv('./tabledbscan', header=True)

res

Finished parsing file /home/ryan/Portfolio/Wine/tabledbscan

Parsing completed. Parsed 26 lines in 0.135892 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,str,str,str,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/ryan/Portfolio/Wine/tabledbscan

Parsing completed. Parsed 26 lines in 0.008952 secs.

HOW MANY WINES IN THECLUSTER ...,mean price,min price,max price,upleft: lat long:,upright: lat long:
31,29.8160715178,10.0,55.0,47.816008 -123.434727,47.816008 174.112313
20,31.9646110683,15.0,240.0,49.94067 -122.871963,49.94067 173.116234
2308,36.6495812285,9.0,600.0,50.022967 -123.498398,50.022967 176.922699
131,35.6204053043,10.0,1700.0,48.874956 -123.207764,48.874956 176.733844
145,33.5438348741,8.99,235.0,49.980794 -123.294335,49.980794 176.922699
12,30.6783440856,14.0,139.0,47.734308 -123.207764,47.734308 139.077086
287,37.4567900807,10.0,287.0,49.988934 -123.487213,49.988934 175.485649
333,34.2632959806,9.0,325.0,49.260232 -123.250455,49.260232 176.870699
3097,35.4352100925,8.0,720.0,50.053387 -123.447497,50.053387 176.922699
14,40.3202768402,13.0,240.0,48.48707 -122.556197,48.48707 15.687983


## Interpretation

Now, the wordclouds showed us that there are still some very common words in each cluster. We could strip out more wore words, and repeat (also play with radius and wieghts), but the notebook's goal is to explore other techniques as well. So let's try to do something , perhaps, more intelligent.

Also, a quick note on the stats, some of the clusters are rather small, which may not be a problem. For example, there are a few sakes in the data, so the sakes could easily be clustered together in a small group. 

Anyway, using Wine Enthusaiast and Wikipedia, a list of important words in wine was created. The notebook will now see if these words are better to look for in the data and use.

In [155]:
words1=gl.SFrame("./words.gl/")

In [156]:
#dropna
words1=words1.dropna()

words1

X1
austere
grown
brilliant
american
viticultural
chewy
volatile
filtering
french
pomegranate


## Regression, an aside

The word list above was collected from wikipedia and wine enthusiast, but perhaps, we can create a list from the data. Using regression, let's regress bag of words on the professional reviews on their wine scores.

In [157]:
#load wine reivew data
rate=gl.SFrame('./rate4model.gl/')
score=rate[['id','abbr', 'score', 'bow']]

score

id,abbr,score,bow
0,TP,92,"{'finish': 1, 'ripe': 1,'cassis': 1, 'lush': 1, ..."
0,RP,94,"{'tannins': 1, 'point':1, 'sweet': 1, 'hard' ..."
0,WE,92,"{'tannins': 1, 'dry': 1,'finish': 1, 'crisp': 1, ..."
1,WW,92,"{'alluring': 2, 'palate':2, 'tasted': 1, 'ripe': ..."
2,V,94,"{'breadth': 1, 'deep': 1,'shape': 1, 'flavors' ..."
2,RP,93,"{'blue': 1, 'spice': 1,'flower': 1, 'mouthfe ..."
2,WE,99,"{'currants': 1, 'ripe':1, 'juicy': 1, 'expect': ..."
2,JS,95,"{'tannins': 2,'currants': 1, 'ripe' ..."
3,JH,95,"{'box': 1, 'fragrant': 1,'palate': 1, 'acidity': ..."
3,RP,90,"{'tannins': 1, 'rating':1, 'cassis': 1, 'garn ..."


In [158]:
#create dataframe for regression
rreg=rate[['score','bow']].unpack('bow', column_name_prefix=None,na_value=0)
rreg.__materialize__()

#get column names
g=rreg.column_names()
#get unpacked column names
g=g[1::]

#construct frame for regression
for i in g:
    rreg[i]=rreg[i].apply(lambda x:0 if x is None else x,skip_undefined=False)
    rreg.__materialize__()

rreg['score']=rreg['score'].apply(lambda x: int(x))

In [159]:
#show frame for reg
rreg

score,bow.absolutely,bow.abundant,bow.accent,bow.accented,bow.accents,bow.accessible,bow.acid,bow.acidity
92,0,0,0,0.0,0,0.0,0,0
94,0,0,0,0.0,0,0.0,0,0
92,0,0,0,0.0,0,0.0,0,0
92,0,0,0,0.0,0,0.0,0,1
94,0,0,0,0.0,0,0.0,0,0
93,0,0,0,0.0,0,0.0,0,0
99,0,0,0,0.0,0,0.0,0,0
95,0,0,0,0.0,0,0.0,0,0
95,0,0,0,0.0,0,0.0,0,1
90,0,0,0,0.0,0,0.0,1,0


<b>Note: The frame is sparse, but we will ignore that issue for now and do the regression anyway.</b>

In [162]:
#drop missing values
rreg1=rreg.dropna('score')
#run regression
model=gl.regression.create(rreg1,target='score', features=g)
coef=model['coefficients']
#get words
words=coef[['name','value']]
#drop bow. 
words['name']=words['name'].apply(lambda x: x[4:])
#sort words by value
words=words.sort('value')
#keep only words that have large impact positive or negative
words['value']=words['value'].apply(lambda x: x if x>5 or x<-5 else None)
#drop na
words=words.dropna('value')
#get words
word=list(words['name'])
#drop intercept
word=word[:-1]

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Linear regression:

--------------------------------------------------------

Number of examples          : 7520

Number of features          : 1163

...

| 6         | 15       | 1.000000  | 3.174211     | 78.107568          | 76.107568            | 27.446539     | 26.464453       |

| 10        | 19       | 1.000000  | 3.798020     | 256.040360         | 208.104119           | 50.181152     | 49.873901       |

+-----------+----------+-----------+--------------+--------------------+----------------------+---------------+-----------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [163]:
#len of word
len(word)

436

In [164]:
#see amount of overlap with list from wiki and WE
len(list(set(word)&set(list(words1['X1']))))

27

## Interpretation

Remember the goal! Cluster/Catergorize! Honestly, you would need to test both sets with different clustering techniques and features and EVEN then you would need to use some subjective measure for what is a good grouping of wines. So here, we are not going say much about which list of words to reduce the tf-idf on would be better. We will use the first list just beause this regression is about scoring wine and I am not a 100% sold that a list created with score in mine caputures the type of clusters we want to (but like I said, we need to test and see).

## K-means

K-means finds cluster centers for a predetermined number of clusters ("K") by minimizing the sum of squared distances from each point to its assigned cluster. Points are assigned to the cluster whose center is closest.

Lloyd's algorithm is the standard way to compute K-means clusters, and it describes the essential intuition for the method. After initial centers are chosen, two steps repeat until the cluster assignment no longer changes for any point (which is equivalent to the cluster centers no longer moving):
<ol>
<li>    Assign each point to the cluster with the closest center.</li>
<li>    Update each cluster center to the be mean of the assigned points.</li></ol>

### Performing K-means here
For this example, we will just run K-means of the tf-idf column. And will not pick K with the standard trick [K = int(sqrt(sf.num_rows() / 2.0))]. For this run, just set K=10

In [165]:
#pick feature
sf1=sf[['tfidf']]
#set words1 as list
x=list(words1['X1'])

#get rid of words that aren't from wiki/we
sf1['tfidf']=sf1['tfidf'].dict_trim_by_keys(x,exclude=False)

#run k-means
kmeans_model = gl.kmeans.create(sf1, num_clusters=10, max_iterations=200)

Batch size is larger than the input dataset. Switching to an exact Kmeans method.

Choosing initial cluster centers with Kmeans++.

+---------------+-----------+

| Center number | Row index |

+---------------+-----------+

...

| 12        | 25                            |

| 13        | 6                             |

| 14        | 2                             |

| 15        | 0                             |

+-----------+-------------------------------+

In [166]:
kmeans_model['cluster_info'][['cluster_id', 'size', 'sum_squared_distance']]

cluster_id,size,sum_squared_distance
0,373,53731.7275276
1,1,0.0
2,355,22889.016849
3,2,260.737915039
4,510,29165.6970606
5,6150,185440.280311
6,514,27680.574976
7,55,6015.23508263
8,33,2702.76658344
9,1461,116765.547257


In [167]:
kmeans_model['cluster_info']

tfidf,cluster_id,size,sum_squared_distance
"{'aeration':0.1685537738374681, ...",0,373,53731.7275276
"{'dry':2.307250071767802, ...",1,1,0.0
"{'aeration':0.0885500811850361, ...",2,355,22889.016849
"{'acidity':0.5958887656160589, ...",3,2,260.737915039
"{'aeration':0.10566480275861437, ...",4,510,29165.6970606
"{'brix':0.005052291935111079, ...",5,6150,185440.280311
"{'aeration':0.034947502857907535, ...",6,514,27680.574976
"{'aeration':0.24495022457678806, ...",7,55,6015.23508263
"{'aroma':0.09302102419024275, ...",8,33,2702.76658344
"{'aeration':0.14139266898911101, ...",9,1461,116765.547257


## Interpretation

Just like in the DBSCAN, we can use wrod clouds to look at the custers and can see how other data falls out in these clusters. The notebook will not produce those here. If interested, try yourself. Ultimately, K-means looks to have created at least some decent size clusters. Let's try Topic models on the wine maker's notes and see how that works.

## Topic Models

"Topic models" are a class of statistical models for text data. These models typically assume documents can be described by a small set of topics, and there is a probability of any word occurring for a given "topic".

For example, suppose we are given the documents shown below, where the first document begins with the text "The sandwich was terrible. I..." and continues with a long description of the eater's woes. A topic model attempts to do two things:
<ol>
<li>  Learn "topics": collections of words that co-occur in a meaningful way </li>
<li>  Learn how much each document pertains to each topic. </li></ol>

### Topic Model here

This notebook is going to create a topic model on a combine winemaker's notes and review column.

In [168]:
#quick space cleaning
sf['wmr']=sf['wmr'].apply(lambda x: x.replace('  ',' '))
sf['wmr']=sf['wmr'].apply(lambda x: x.replace('   ',' '))
sf['wmr']=sf['wmr'].apply(lambda x: x.replace('    ',' '))

#strip winemaker s notes
sf['wmr']=sf['wmr'].apply(lambda x: x.replace('winemaker s notes',''))

In [169]:
#create bag of words on wmr
sf['bow']=gl.text_analytics.count_words(sf['wmr'])

#trim common words
sf['bow']=sf['bow'].dict_trim_by_keys(gl.text_analytics.stopwords(), exclude=True)

In [170]:
sf[['bow']]

bow
"{'decade': 1, 'style': 2,'lariat': 2, 'cherry' ..."
"{'alluring': 2, 'firm':1, 'palate': 3, 'tast ..."
"{'breadth': 1,'currants': 2, ..."
"{'tannins': 2, 'rating':1, 'palate': 1, ..."
"{'currants': 2,'aeration': 1, 'blend ..."
"{'precise': 1, 'summer':1, 'currants': 2, ..."
"{'tannins': 3, 'rating':1, 'palate': 1, ..."
"{'traditions': 1,'aromatics': 2, 'years': ..."
"{'tannins': 2, 'body': 1,'currants': 1, 'palate': ..."
"{'years': 2, 'leads': 1,'impressively': 1, ..."


In [171]:
#create topic model
model = gl.topic_model.create(sf['bow'],num_iterations=50)

Learning a topic model

Number of documents      9454

Vocabulary size     20916

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

...

| 20        | 837.014ms     | 1.33381e+07    | 0               |

| 30        | 1.21s         | 1.44908e+07    | 0               |

| 40        | 1.54s         | 1.51436e+07    | 0               |

| 50        | 1.90s         | 1.52619e+07    | 0               |

+-----------+---------------+----------------+-----------------+

In [172]:
print model.get_topics()

+-------+----------+-----------------+
| topic |   word   |      score      |
+-------+----------+-----------------+
|   0   | tannins  | 0.0320851751481 |
|   0   | licorice | 0.0178568795652 |
|   0   |   full   | 0.0171344065253 |
|   0   |  finish  | 0.0145836343638 |
|   0   |  years   | 0.0143624691475 |
|   1   |   red    | 0.0494204236283 |
|   1   |  cherry  | 0.0446842492818 |
|   1   |  palate  | 0.0326709603371 |
|   1   |  spice   | 0.0293348959252 |
|   1   |  black   | 0.0248061452726 |
+-------+----------+-----------------+
[50 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [173]:
print model.get_topics(output_type='topic_words')

+-------------------------------+
|             words             |
+-------------------------------+
| [tannins, licorice, full, ... |
| [red, cherry, palate, spic... |
| [wine, fruit, full, bodied... |
| [oak, vineyard, pinot, fla... |
| [fruit, flavors, notes, ar... |
| [tannins, blend, sauvignon... |
| [wine, cabernet, made, win... |
| [acidity, white, finish, n... |
| [wine, fresh, chardonnay, ... |
| [palate, red, ripe, aromas... |
+-------------------------------+
[10 rows x 1 columns]



In [174]:
sf['topic']=model.predict(sf['bow'])

#make a table to see its performance
test=model.get_topics(output_type='topic_words')
a=[]
i=0
while i<10:
    a.append(len(sf['name'][sf['topic']==i]))
    i+=1
a
test['in_cat']=a
test

words,in_cat
"[tannins, licorice, full,finish, years] ...",787
"[red, cherry, palate,spice, black] ...",1139
"[wine, fruit, full,bodied, cabernet] ...",405
"[oak, vineyard, pinot,flavors, palate] ...",660
"[fruit, flavors, notes,aromas, cherry] ...",886
"[tannins, blend,sauvignon, black, mer ...",1016
"[wine, cabernet, made,wines, grapes] ...",853
"[acidity, white, finish,notes, fresh] ...",1212
"[wine, fresh, chardonnay,crisp, light] ...",1717
"[palate, red, ripe,aromas, rich] ...",779


## Interpretation

Similar to what we said about K-means, we would run word clouds and look at othr data to judge how god the clustering is. This notebook will leave that to the reader. Anyway, we see a decent breakdown in the clustering, in terms of no cluster is super smallm(though that doesn't actual mean much).

## DBSCAN, a last time, with WE and Wiki filtering on tf-idf

In [175]:
#load data frame
cur1=gl.SFrame('./4modelclean022316/')

cur1

id,year,lat,long,ln_price,bow
0,2013,38.220417,-122.228952,4.09434456222,"{'decade': 1, 'style': 2,'lariat': 2, 'cherry' ..."
1,2014,38.546127,-122.49276,3.58351893846,"{'alluring': 2, 'firm':1, 'palate': 3, 'tast ..."
2,2012,43.225883,10.563555,5.45958551414,"{'breadth': 1,'currants': 2, ..."
3,2010,-34.919913,138.654506,3.80666248977,"{'tannins': 2, 'rating':1, 'palate': 1, ..."
4,2011,38.741804,-122.955029,3.80666248977,"{'currants': 2,'aeration': 1, 'blend ..."
5,2000,45.224217,-0.772444,3.96025104452,"{'precise': 1, 'summer':1, 'currants': 2, ..."
6,2013,38.387141,-122.313005,3.63758615973,"{'tannins': 3, 'rating':1, 'palate': 1, ..."
7,2010,43.077656,11.491754,4.31748811354,"{'traditions': 1,'aromatics': 2, 'years': ..."
8,2012,38.529416,-122.490612,4.27666611902,"{'tannins': 2, 'body': 1,'currants': 1, 'palate': ..."
9,2012,38.398864,-122.325167,5.1059454739,"{'years': 2, 'leads': 1,'impressively': 1, ..."


In [176]:
#filter with WE and wiki list
cur1['bow']=cur1['bow'].dict_trim_by_keys(x, exclude=False)

In [177]:
#create tfodf column
cur1['tfidf']=gl.text_analytics.tf_idf(cur1['bow'])
cur1=cur1.remove_columns(['bow'])

In [178]:
#create sframe for dbscan
cur=cur1[['year','lat', 'long', 'ln_price', 'tfidf']]

#set distances to be used
comp_dis=[[['lat', 'long'], 'euclidean', 1],[['ln_price', 'year'], 'euclidean', .5],[['tfidf'], 'cosine', 2]]

#run dbscan
dbscan_model = gl.dbscan.create(cur,distance=comp_dis, radius=5)
dbscan_model.summary()

Starting brute force nearest neighbors model training.

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

...

| 50147                       |

| 0                           |

+-----------------------------+

[INFO] graphlab.toolkits.clustering.dbscan: Processing boundary points.


Class                                   : DBSCANModel

Schema
------
Number of examples                      : 6504
Number of feature columns               : 5
Max distance to a neighbor (radius)     : 5
Min number of neighbors for core points : 10
Number of distance components           : 3

Training summary
----------------
Total training time (seconds)           : 12.1917
Number of clusters                      : 11

Accessible fields
-----------------
cluster_id                              : Cluster label for each row in the input dataset.


In [179]:
#quick and dirty data manipulation

cur2=gl.SFrame('./formodel.gl/')
test1=cur2[['id', 'name']]
test1
test1=test1.add_row_number('row_id')
cur=cur.add_row_number('row_id')
cur=cur.join(test1, on='row_id', how='left')
cur=cur.join(dbscan_model['cluster_id'], on='row_id', how='left')
cur[['id','name','cluster_id', 'type']].head()

id,name,cluster_id,type
0,Jamieson Ranch VineyardsDouble Lariat Cabernet ...,8,core
1,Rombauer Chardonnay 2014,8,core
2,Tenuta San GuidoSassicaia 2012 ...,2,core
3,Clarendon Hills SandownCabernet Sauvignon 2010 ...,6,core
4,Silver Oak AlexanderValley Cabernet Sauvi ...,8,core
5,Chateau Lafite-Rothschild2000 ...,2,core
6,Clos du Val CabernetSauvignon 2013 ...,8,core
7,Valdicava Brunello diMontalcino 2010 ...,2,core
8,Duckhorn Napa ValleyCabernet Sauvignon 2012 ...,8,core
9,Stag's Leap Wine CellarsCask 23 Cabernet ...,8,core


In [180]:
#wordcloud function
def plot_wordcount(cluster_id, model):
        from wordcloud import WordCloud
        cluster_data = cur.filter_by(model['cluster_id']['row_id'][model['cluster_id']['cluster_id']==cluster_id],'row_id')
        #print cluster_data['tfidf'].sketch_summary()
        #print cluster_data['name']
        data = list(cluster_data['tfidf'])
        data = [' '.join(c) for c in data]
        data = ' '.join(data)
        wc = WordCloud().generate(data) 
        figure(figsize=(20,10))
        return imshow(wc)
    
#creat word clouds
i=0
while i<11:
        plot_wordcount(i,dbscan_model)
        plt.savefig('termedcluster'+str(i)+'png')
        i+=1

## Interpretation
 
<center><table>
<tr><td><img src="termedcluster0png.png"></td>
<td><img src="termedcluster1png.png"></td>
<td><img src="termedcluster2png.png"></td></tr><br>
<tr><td><img src="termedcluster3png.png"></td>
<td><img src="termedcluster4png.png"></td>
<td><img src="termedcluster5png.png"></td></tr><br>
<tr><td><img src="termedcluster6png.png"></td>
<td><img src="termedcluster7png.png"></td>
<td><img src="termedcluster8png.png"></td></tr><br>
<tr><td><img src="termedcluster9png.png"></td>
<td><img src="termedcluster10png.png"></td>
</tr><br>
</table></center>


In [ ]:
#create table for info on dbscan
import csv
filenamed= 'tabledbscanterms'
titles=['HOW MANY WINES IN THE CLUSTER','mean price', 'min price', 'max price','upleft: lat long:', 'upright: lat long: ','downleft: lat long: ', 'downright: lat long: ']

writer = csv.writer(open(filenamed, 'a'))
writer.writerow(titles)   
 
#to get min, max, mean price
def getstats(cluster_id,model):
     cd = cur.filter_by(model['cluster_id']['row_id'][model['cluster_id']['cluster_id']==cluster_id],'row_id')
     holder=[]
     holder.append(str(len(cd['lat'])))
     holder.append(str(math.exp(cd['ln_price'].mean())))
     holder.append(str(math.exp(cd['ln_price'].min())))
     holder.append(str(math.exp(cd['ln_price'].max())))
     holder.append(str(cd['lat'].max()) +' '+str(cd['long'].min()))
     holder.append(str(cd['lat'].max()) +' '+str(cd['long'].max()))
     holder.append(str(cd['lat'].min()) +' '+str(cd['long'].min()))
     holder.append(str(cd['lat'].min()) +' '+str(cd['long'].max()))
     writer = csv.writer(open(filenamed, 'a'))
     writer.writerow(holder)   
        
#to loop through clusters
i=0
while i<11:
    getstats(i,dbscan_model)
    i+=1

In [181]:
#load table results
rs=gl.SFrame.read_csv('./tabledbscanterms', header=True)

rs

Finished parsing file /home/ryan/Portfolio/Wine/tabledbscanterms

Parsing completed. Parsed 11 lines in 0.068726 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,float,float,float,str,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/ryan/Portfolio/Wine/tabledbscanterms

Parsing completed. Parsed 11 lines in 0.006967 secs.

HOW MANY WINES IN THECLUSTER ...,mean price,min price,max price,upleft: lat long:,upright: lat long:
31,45.5948182517,12.99,52.4704967264,40.811024 22.087299,40.811024 25.46019
20,36.6763625502,14.0,75.0,-44.693447 168.949127,-44.693447 169.307707
2308,49.5785038605,8.99,1700.0,50.053387 -9.018866,50.053387 18.422677
131,29.1834732411,10.0,225.0,-33.382953 18.380004,-33.382953 22.70694
145,28.1886699278,13.0,115.0,-36.755802 172.611412,-36.755802 176.922699
12,52.8605372684,15.0,145.0,37.581517 -6.3557,37.581517 -4.644164
287,38.6442001737,9.0,530.0,-33.833028 138.524613,-33.833028 146.214502
333,29.4974994737,10.0,235.0,-30.027191 -71.548389,-30.027191 -68.298705
3097,36.5032490076,10.0,600.0,47.80473 -123.498398,47.80473 -117.115283
14,43.3824906733,17.0,89.0,-33.824053 115.040998,-33.824053 115.065145


## Conclusions

Well that was fun, but as you can see. We only scratched the service. To do a problem like this justice, one needs to decide on a measuring stick. <b> What does it mean for wines to be clustered well?</b> Once you decide on that, you need explore many different permutations of the models we looked at, think deeply about feature engineering, ponder on ways to discard models that won't help earlier rather than later. 

Anyway, as always, Happy Exploring.